<a href="https://colab.research.google.com/github/PGM-Lab/PAC2BAYES/blob/master/PAC_MNIST_ExponentialDistribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set-Up



In [ ]:
import datetime
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D

from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam



from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
assert(tf.test.gpu_device_name())
tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(True) # Enable XLA.

# Data

In [ ]:
num_classes = 10
train_size = 1000

def load_data(data_set, train_size):
  (x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
  #(x_test, y_test), (x_train, y_train) = datasets.mnist.load_data()
  
  tf.keras.utils.set_random_seed(123)
  a = np.random.permutation(train_size)
  x_train = x_train[a,...]
  y_train = y_train[a]
  # Add a new axis
  x_train = x_train[..., np.newaxis]
  x_test = x_test[..., np.newaxis]


  # Convert class vectors to binary class matrices.
  y_train = to_categorical(y_train, num_classes)
  y_test = to_categorical(y_test, num_classes)

  # Data normalization
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train /= 255
  x_test /= 255

  return x_train, y_train, x_test, y_test

Load Fashion MNIST

In [ ]:
x_train, y_train, x_test, y_test = load_data('mnist', train_size)

In [ ]:
print("Training samples:", x_train.shape[0])
print("Test samples:", x_test.shape[0])

# Loss Function

In [ ]:
from tensorflow.keras.callbacks import Callback
import warnings

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [ ]:
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten

def get_lenet(l2_reg = 0.0):
    tf.keras.utils.set_random_seed(123)

    LeNet_l2 = Sequential()
    LeNet_l2.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1), 
                kernel_regularizer=tf.keras.regularizers.L2(l2_reg),
                bias_regularizer=tf.keras.regularizers.L2(l2_reg)))
    LeNet_l2.add(MaxPool2D(strides=2))
    LeNet_l2.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu', 
                kernel_regularizer=tf.keras.regularizers.L2(l2_reg),
                bias_regularizer=tf.keras.regularizers.L2(l2_reg)))
    LeNet_l2.add(MaxPool2D(strides=2))
    LeNet_l2.add(Flatten())
    LeNet_l2.add(Dense(256, activation='relu', 
                kernel_regularizer=tf.keras.regularizers.L2(l2_reg),
                bias_regularizer=tf.keras.regularizers.L2(l2_reg)))
    LeNet_l2.add(Dense(84, activation='relu',
                kernel_regularizer=tf.keras.regularizers.L2(l2_reg),
                bias_regularizer=tf.keras.regularizers.L2(l2_reg)))
    LeNet_l2.add(Dense(10))

    LeNet_l2.build()
    cce = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True
    )


    adam = Adam(learning_rate=1e-3)
    LeNet_l2.compile(loss=cce, metrics=[cce, 'accuracy'], optimizer=adam)
    return LeNet_l2

# Training Set-up

In [ ]:
def model_norm(model):
    norm = []
    for w in model.get_weights():
        norm.append(tf.norm(w))
    return np.sum(norm)

def model_relative_norm(model, model_reference):
    norm = []
    for i in range(len(model.get_weights())):
        norm.append(tf.norm(model.get_weights()[i]-model_reference.get_weights()[i]))
    return np.sum(norm)    

In [ ]:
def train_and_evaluate(model, epochs, batch_size=train_size):
    callbacks = [EarlyStoppingByLossVal(monitor='categorical_crossentropy', value=0.05, verbose=1)]
    model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, callbacks = callbacks, verbose = 0)
    train_metrics = model.evaluate(x_train, y_train)
    test_metrics = model.evaluate(x_test, y_test)
    return train_metrics, test_metrics

In [ ]:
def eval_jensen(model, lambdas):
    cce_red = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.keras.losses.Reduction.NONE
    )
    y_pred = model.predict(x_test)
    log_p = -cce_red(y_test, y_pred)
    return np.array([(tfp.math.reduce_logmeanexp(lamb * log_p) - tf.reduce_mean(lamb * log_p)) for lamb in lambdas])

def jensen_function(model, lamb):
    cce_red = tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, reduction=tf.keras.losses.Reduction.NONE
    )
    y_pred = model.predict(x_test)
    log_p = -cce_red(y_test, y_pred)
    return (tfp.math.reduce_logmeanexp(lamb * log_p))/lamb, tf.reduce_mean(log_p), (tfp.math.reduce_logmeanexp(lamb * log_p) - tf.reduce_mean(lamb * log_p))/lamb



def rate_function(model, lambdas, s_values):
  jensen_vals = eval_jensen(model, lambdas)
  def temp(s):
    if s>0:
      return np.max(lambdas[lambdas>0]*np.abs(s) - jensen_vals[lambdas>0])
    else:
      return np.max((-lambdas[lambdas<0])*np.abs(s) - jensen_vals[lambdas<0])
  
  return [temp(s) for s in s_values]

def rate_function_positive(model, lambdas, s_values):
  jensen_vals = eval_jensen(model, lambdas)

  return [ np.max(lambdas*s - jensen_vals) for s in s_values]


def rate_function_negative(model, lambdas, s_values):
  jensen_vals = eval_jensen(model, -lambdas)

  return [ np.max(lambdas*s - jensen_vals) for s in s_values]


def inverse_rate_function_alpha(model, lambdas, alpha_vals):
  jensen_vals = eval_jensen(model, lambdas)

  out = []
  for alpha in alpha_vals:
    vec = (jensen_vals + alpha)/lambdas
    index = np.argmin(vec)
    if index==len(lambdas)-1:
      raise Exception('Not enough high lambda range')

    out.append(vec[index])
  return out


def inverse_rate_function_alpha_negative(model, lambdas, alpha_vals):
  jensen_vals = eval_jensen(model, -lambdas)

  out = []
  for alpha in alpha_vals:
    vec = (jensen_vals + alpha)/lambdas
    index = np.argmin(vec)
    if index==len(lambdas)-1:
      raise Exception('Not enough high lambda range')

    if index==0:
      raise Exception('Not enough low lambda range')

    out.append(vec[index])
  return out


def inverse_rate_function(model, model_reference, lambdas, n_samples):
  jensen_vals = eval_jensen(model, lambdas)
  model_class_capacity = np.log((model_relative_norm(model, model_reference)+0.05)/0.05)*model.count_params()/n_samples

  inverse_rate_vals = (jensen_vals+model_class_capacity+np.log(lambdas.size/0.05))/lambdas

  return np.min(inverse_rate_vals), lambdas[np.argmin(inverse_rate_vals)]

def model_weights(model):
    norm = []
    for w in model.get_weights():
        norm.append(tf.reshape(w,[-1]))

    w_vals = []
    for w in norm:  
      w_vals.append(tf.reshape(w,[-1]))

    return tf.concat(w_vals,axis=0)



In [ ]:
l2_values = [0.0, 0.0, 0.01, 0.01]
bach_sizes = [train_size, 50, train_size, 50]
labels = ['GD', 'SGD', 'GD+L2', 'SGD+L2']
#l2_values = [0.0]

# Train

In [ ]:
models = [get_lenet(l2) for l2 in l2_values]

In [ ]:
metrics = [train_and_evaluate(models[i], 200, batch_size=bach_sizes[i]) for i in range(len(models))]

# Plot Jensen-Gap and Rate Functions

In [ ]:
norms = [model_norm(model) for model in models]
norms

In [ ]:
lambdas = np.linspace(-0.5, 0.5, 100)
jensens = [eval_jensen(model, lambdas) for model in models]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

plt.rcParams['figure.figsize'] = [10, 5]

In [ ]:
for i in np.arange(len(l2_values)):
    plt.plot(lambdas, jensens[i], label = "{}".format(labels[i]))
    plt.ylim(0,0.1)
plt.legend()
plt.show()

# Sampling $\alpha$ for a given model using data sets of size $batch\_size$

In [ ]:
#We fix the i-th
i=1
batch_size=50


batch_vals = []

#L(\theta)
eval_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
eval_dataset = eval_dataset.shuffle(buffer_size=1024).batch(batch_size)

metric_test = models[i].evaluate(x_test, y_test, batch_size=batch_size)
L=metric_test[1]

for step, (x_batch, y_batch) in enumerate(eval_dataset):
  #\hatL(D_i,\theta)
  metric_batch = models[i].evaluate(x_batch, y_batch, batch_size=batch_size, verbose=False)
  batch_vals.append(metric_batch[1])

batch_vals = np.array(batch_vals)


In [ ]:
print(L)
print(np.mean(batch_vals))
plt.hist(batch_vals, bins=20)
batch_vals.shape

In [ ]:
np.min(batch_vals)

In [ ]:
lambdas = np.arange(-5, 5, 0.01)

total_alpha_vals = rate_function(models[i],lambdas,L-batch_vals)
total_alpha_vals = batch_size*np.array(total_alpha_vals)*np.sign(L-batch_vals)
np.sum(total_alpha_vals)

We have to distinguish between positive and negative alphas

In [ ]:
alpha_vals = total_alpha_vals[L>batch_vals]
print(len(alpha_vals))
alpha_vals = alpha_vals[alpha_vals>np.percentile(alpha_vals, 20)]#-0.05
len(alpha_vals)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

lambda_rate = 1#/np.mean(alpha_vals) #batch_size
print(1/np.mean(alpha_vals))


# Plot histogram
plt.hist(alpha_vals, bins = 20, density=True)
#plt.hist(data, bins=30, density=True, alpha=0.5)

# Plot density
x = np.linspace(0, np.log(1.0/0.0001)/lambda_rate, 1000)
y = lambda_rate *np.exp(-lambda_rate * x)
plt.plot(x, y, color='red')
#plt.plot(-x, y, color='red')

# Add labels and title
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Exponential Distribution')

# Show the plot
plt.show()


In [ ]:
from scipy.stats import skew, kurtosis
print(skew(alpha_vals))
print(kurtosis(alpha_vals))

In [ ]:
import numpy as np
from scipy.stats import expon, kstest, anderson

# Define rate parameter
rate = 1#/np.mean(alpha_vals)
print(1/np.mean(alpha_vals))


# Perform Kolmogorov-Smirnov test
ks_statistic, p_value = kstest(alpha_vals, expon.cdf, args=(0, 1/rate))
print('KS test statistic:', ks_statistic)
print('KS test p-value:', p_value)

# Perform Anderson-Darling test
ad_statistic, critical_values, significance_levels = anderson(alpha_vals, 'expon')
print('AD test statistic:', ad_statistic)
print('AD critical values:', critical_values)
print('AD significance levels:', significance_levels)
